# Music Genre Classification

## Data Pre-Processing for Recurrent NN

Fabrizio Niro - Jacopo Signò

GTZAN Dataset - Music Genre Classification 

https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification

In [5]:
import librosa as lr
import pandas as pd
import numpy as np
import IPython.display as ipyd
from collections import Counter
import math
import os
from pathlib import Path
import pickle
import torch
import sklearn
import torch.nn.functional as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

The function "tracksplit" splits the audio track, loaded into an array, in n subtracks

In [6]:
def tracksplit(track, nsplit = 10):
    #splits a track in n subtracks of equal length
    length = len(track) // nsplit
    subtracks = []
    acc = 0

    for i in range(nsplit):
        subtracks.append(track[acc:acc+length])
        acc += length

    return subtracks

We iterate through the dataset structure to split all the audio tracks into 10 tracks of 3 seconds length

In [7]:
sec3_subtracks = []

for directory in os.listdir('Data/genres_original'):

    for file in os.listdir('Data/genres_original/{}'.format(directory)):

        try:
            track, _ = lr.load('Data/genres_original/{}/{}'.format(directory, file))

        except:
            track, _ = lr.load('Data/genres_original/{}/{}{}'.format(directory, file[:-5], '0.wav'))

        for subtr in tracksplit(track):

            sec3_subtracks.append(subtr)


In [8]:
sec3_len =[]
for i in sec3_subtracks:
    sec3_len.append(len(i))

Counter(sec3_len).keys()

dict_keys([66179, 66924, 66150, 66176, 67580, 66770, 66000, 66968, 66140, 66440, 66814, 66528, 66167, 67228, 66308, 67012, 66134, 66352, 66946, 66792, 67034, 66616, 66418, 66506, 66682, 66198, 67210, 66748, 66594, 66880, 66110, 66374, 66330])

In [9]:
min_len = min(Counter(sec3_len).keys())
min_len

66000

In [56]:
sec3_subtr_trim = []
for i in sec3_subtracks:
    sec3_subtr_trim.append(i[:min_len])

sec3_len_trim =[]
for i in sec3_subtr_trim:
    sec3_len_trim.append(len(i))

Counter(sec3_len_trim).keys()

dict_keys([66000])

In [12]:
Path('Processed_data/RNN').mkdir(parents=True, exist_ok=True)

In [13]:
with open('Processed_data/sec3_subtr_trim', 'wb') as f:
    pickle.dump(sec3_subtracks, f)

The pandas dataframe "data" stores for each row the track id, the label and the array of the splitted tracks

In [14]:
sec3_track_id = []
label = []

for dir in os.listdir('Data/genres_original'):

    for i in range(1000):

        sec3_track_id.append(f'{dir}_{i}')
        label.append(f'{dir}')

In [15]:
data = pd.DataFrame()
data['sec3_track_id'] = sec3_track_id
data['label'] = label
data['sec3_subtracks'] = sec3_subtr_trim

In [16]:
with open('Processed_data/data_id_lab_subt', 'wb') as f:
    pickle.dump(data, f)

In [17]:
with open('Processed_data/data_id_lab_subt', 'rb') as f:
    data = pickle.load(f)

In [18]:
data

,sec3_track_id,label,sec3_subtracks
0,hiphop_0,hiphop,"[-0.14361572, -0.13696289, -0.14282227, -0.153..."
1,hiphop_1,hiphop,"[-0.021484375, -0.03277588, -0.059570312, -0.0..."
2,hiphop_2,hiphop,"[-0.18191528, -0.046417236, -0.0680542, -0.227..."
3,hiphop_3,hiphop,"[0.016571045, 0.037506104, 0.05117798, 0.03680..."
4,hiphop_4,hiphop,"[-0.12823486, -0.1592102, -0.18588257, -0.1886..."
...,...,...,...
9995,country_995,country,"[-0.04550171, -0.15084839, -0.2517395, -0.2762..."
9996,country_996,country,"[0.07611084, 0.07437134, 0.06213379, 0.0467529..."
9997,country_997,country,"[-0.120269775, -0.13812256, -0.07296753, -0.06..."
9998,country_998,country,"[-0.043762207, -0.024261475, -0.033416748, -0...."


We compute the Mel Frequency C Coef. for each splitted track. It will be the feature to feed the network with

In [19]:
sec3_mfcc = []

for i in range(len(data['sec3_subtracks'])):
    
    sec3_mfcc.append(lr.feature.mfcc(y=data['sec3_subtracks'][i]))

In [20]:
with open('Processed_data/sec3_mfcc', 'wb') as f: 
    pickle.dump(sec3_mfcc, f)

In [21]:
with open('Processed_data/sec3_mfcc', 'rb') as f:
    sec3_mfcc = pickle.load(f)

We store the MFCC matrices of all the splitted tracks into a torch tensor

In [41]:
sec3_mfcc = np.array(sec3_mfcc, dtype='float64')

In [42]:
sec3_mfcc = torch.tensor(sec3_mfcc)

We create the target array encoding the labels with One Hot method

In [58]:
labels = np.array(data['label'])
le = preprocessing.LabelEncoder()
target = le.fit_transform(labels)
target = torch.tensor(target)
#target_one_hot = tf.one_hot(target.to(torch.int64), 10)

In [59]:
target

tensor([4, 4, 4,  ..., 2, 2, 2])

In [60]:
#target_one_hot

In [61]:
with open('Processed_data/RNN/target', 'wb') as f: 
    pickle.dump(target, f)

In [62]:
with open('Processed_data/RNN/target', 'rb') as f:
    target = pickle.load(f)

In [63]:
X = torch.flatten(sec3_mfcc, start_dim=1)

y = target

In [64]:
X

tensor([[-208.5672, -218.5552, -192.0441,  ...,   -2.1380,   -5.1627,
           -7.3314],
        [-127.1170,  -86.5377,  -65.2389,  ...,  -14.1653,   -5.3262,
           -0.6355],
        [-112.1303, -108.4033, -131.5106,  ...,   -9.1636,   -8.7821,
           -6.4954],
        ...,
        [ -98.2837,  -78.3940,  -77.5690,  ...,   -5.3651,   -3.2890,
           -0.2495],
        [ -93.2953,  -91.5130, -120.2912,  ...,  -18.1641,  -15.9425,
          -10.4376],
        [ -50.1972,  -28.8588,  -39.3822,  ...,  -11.4541,  -11.4391,
          -10.4392]], dtype=torch.float64)

In [65]:
y

tensor([4, 4, 4,  ..., 2, 2, 2])

In [66]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [67]:
X_scaled

array([[-0.47011889, -0.6995086 , -0.38556404, ..., -0.12976658,
        -0.44513298, -0.72378191],
       [ 0.29348556,  0.46384707,  0.67899055, ..., -1.44323478,
        -0.46296635,  0.08377488],
       [ 0.43398751,  0.27116436,  0.12262683, ..., -0.89701777,
        -0.83991837, -0.6229539 ],
       ...,
       [ 0.5638002 ,  0.53561027,  0.57547685, ..., -0.48219117,
        -0.24076121,  0.13032775],
       [ 0.61056695,  0.42000365,  0.21681596, ..., -1.87992993,
        -1.62093248, -1.09839215],
       [ 1.01461699,  0.97212047,  0.896063  , ..., -1.14715433,
        -1.12972811, -1.09858412]])

In [68]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.33, random_state=42)

In [69]:
print(X_train.shape, y_train.shape)

(6700, 2580) torch.Size([6700])


In [70]:
print(X_test.shape, y_test.shape)

(3300, 2580) torch.Size([3300])


In [71]:
with open('Processed_data/RNN/X_train', 'wb') as f: 
    pickle.dump(X_train, f)

with open('Processed_data/RNN/y_train', 'wb') as f: 
    pickle.dump(y_train, f)

with open('Processed_data/RNN/X_test', 'wb') as f: 
    pickle.dump(X_test, f)

with open('Processed_data/RNN/y_test', 'wb') as f: 
    pickle.dump(y_test, f)